In [10]:
import itertools
import json
import pickle
import re

import numpy as np
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

from tqdm.notebook import tqdm
tqdm.pandas()

In [11]:
with open("../../data/pseudowords/CoMaPP_all.json") as json_file:
    data = json.load(json_file)
    
# d+1 since we are not replacing the ke-lex this time!
data = [{"example": d["target1"], "cue": " ".join(d["target1"].split()[:d["query_idx"]+1]), "pseudoword": d["label"]} for d in data if d["target1"].split()[d["query_idx"]] in d["label"]]
df = pd.DataFrame.from_dict(data).drop_duplicates(ignore_index=True)
df

,example,cue,pseudoword
0,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,am488
1,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,wenigsten488
2,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,am488
3,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,wenigsten488
4,Unter den Einwohnern der großen EU - Länder si...,Unter den Einwohnern der großen EU - Länder si...,am488
...,...,...,...
8503,"Protestzüge wälzen sich durch München , Eisner...",Protestzüge wälzen sich,sich1573
8504,Und in Sands bewege sich ja einiges .,Und in Sands bewege sich,sich1573
8505,Ulrike beugt sich über die Partitur .,Ulrike beugt sich,sich1573
8506,"Am Anfang gab es viel Widerstand , man holpert...","Am Anfang gab es viel Widerstand , man holpert...",sich1573


In [12]:
df['index'] = df['pseudoword'].str.extract('(\d+)').astype(int)
df.set_index('index', inplace=True)

df

,example,cue,pseudoword
index,,,
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,am488
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,wenigsten488
488,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,am488
488,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,wenigsten488
488,Unter den Einwohnern der großen EU - Länder si...,Unter den Einwohnern der großen EU - Länder si...,am488
...,...,...,...
1573,"Protestzüge wälzen sich durch München , Eisner...",Protestzüge wälzen sich,sich1573
1573,Und in Sands bewege sich ja einiges .,Und in Sands bewege sich,sich1573
1573,Ulrike beugt sich über die Partitur .,Ulrike beugt sich,sich1573


In [13]:
contextleft = pd.read_pickle("../../data/pseudowords/contextleft_text.pickle")

def update_cue(row):
    output = row[['example', 'cue']]
    if len(row['cue'].split()) == 1:  # if the string in cue is empty (except for the kelex)
        # match the index of row with contextleft['construction_id'] and match contextleft['text'] with row['example'] and create matching_entry
        matching_entry = contextleft.loc[(contextleft['construction_id'] == row.name) & (contextleft['text'] == row['example']), 'contextleft'].tolist()
        if len(matching_entry) > 0:
            # Add the left context to the example and to the cue:
            output = [matching_entry[0] + " " + row['example'], matching_entry[0] + " " + row['cue']]
    return output

# Add the left context if there is no cue up until the pseudoword.
#df[["example", "cue"]] = df.apply(update_cue, axis=1)
df

,example,cue,pseudoword
index,,,
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,am488
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,wenigsten488
488,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,am488
488,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,wenigsten488
488,Unter den Einwohnern der großen EU - Länder si...,Unter den Einwohnern der großen EU - Länder si...,am488
...,...,...,...
1573,"Protestzüge wälzen sich durch München , Eisner...",Protestzüge wälzen sich,sich1573
1573,Und in Sands bewege sich ja einiges .,Und in Sands bewege sich,sich1573
1573,Ulrike beugt sich über die Partitur .,Ulrike beugt sich,sich1573


In [14]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'construction'}, inplace=True)

result_df = df.groupby(['construction', 'pseudoword']).agg({'example': list, 'cue': list})

result_df

example  \
construction pseudoword                                                      
5            Und5        [Und schon gar nicht mit der Mehrwertsteuer .,...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  
construction pseudoword                                                     
5            Und5        [Und, Und, Und, Und, Und, Und, Und, Und, Und, ...  
             erst5       [Trainer Lucien Favre hatte schon seine beiden...  
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...  
             nicht5      [Es hat Afghanistan nicht, Dass es in der Fami...  
             recht5      [Trainer Lucien Favre hatte schon seine beiden...  
...                                                                    ...  
1884         Gold1884    [Schweigen ist Silber , reden ist Gold, Schwei...  
             Silber1884  [Schweigen ist Silber, Schweigen ist Silber, A...  
             ist1884     [Schweigen ist, Schweigen ist, Angeben ist, Sc...  
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...  
1987         sehr1987    [Solche Menschen können sich sehr, " " Das hat...  

[506 rows x 2 columns]

In [15]:
with open("../../out/definitions.pickle", "rb") as definitions_file:
    definitions = pd.DataFrame.from_dict(pickle.load(definitions_file), orient="index", columns=["definition"])
    
definitions

,definition
677,"Bei der Konstruktion ""Additiv_Koordinativkompo..."
563,"Bei der ""Gegenüberstellung:V2_V2-Konstruktion""..."
696,"Die ""Konjunktiv:V.conj-I-Konstruktion"" gehört ..."
488,"Die ""Superlativ:am_meisten/wenigstenADJ-Konstr..."
674,"Die ""Kausaler_Konnektor:weil-Konstruktion"" die..."
...,...
875,"Bei der Konstruktion (Kxn) ""Kategorisierung_Tr..."
1323,Die meist als Prädikativum verwendete Konstruk...
85,"Die ""Reduplikation_Quantifizierung:N1_über_N1-..."
1792,"Bei der Konstruktion (Kxn) ""Prädikation_Negati..."


In [16]:
examples = pd.merge(result_df, definitions, how="inner", left_on="construction", right_index=True)
examples

example  \
construction pseudoword                                                      
5            Und5        [Und schon gar nicht mit der Mehrwertsteuer .,...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  \
construction pseudoword                                                      
5            Und5        [Und, Und, Und, Und, Und, Und, Und, Und, Und, ...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht, Dass es in der Fami...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold, Schwei...   
             Silber1884  [Schweigen ist Silber, Schweigen ist Silber, A...   
             ist1884     [Schweigen ist, Schweigen ist, Angeben ist, Sc...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr, " " Das hat...   

                                                                definition  
construction pseudoword                                                     
5            Und5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             erst5       Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             gar5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             nicht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             recht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
...                                                                    ...  
1884         Gold1884    Die "Intensivierung_Komparativ:Xist_SilberYist...  
             Silber1884  Die "Intensivierung_Komparativ:Xist_SilberYist...  
             ist1884     Die "Intensivierung_Komparativ:Xist_SilberYist...  
1986         kaum1986    Die Konstruktion "Relativierung:kaumADJ" gehör...  
1987         sehr1987    Die Konstruktion "Intensivierung:sehrV" gehört...  

[506 rows x 3 columns]

##### Generieren neuer Sätze:

Load the vanilla mbart model:

In [17]:
model = MBartForConditionalGeneration.from_pretrained(
    "facebook/mbart-large-50", return_dict=True
) 
tokenizer = MBart50TokenizerFast.from_pretrained(
    "facebook/mbart-large-50", src_lang="de_DE", tgt_lang="de_DE"
)
model.to("cuda:1")
model.model.encoder.embed_tokens

Embedding(250054, 1024, padding_idx=1)

Complete the cues:

In [18]:
def complete_cues(row):
    try:
        output_texts = []
        scores = []
        for cue, example in zip(row["cue"], row["example"]):
            input_text = "</s> " + cue + " <mask> . </s> de_DE"
            
            target_length = int(1.5 * len(example))  # allow double the length of the original sentence
            
            input_ids = tokenizer([input_text], add_special_tokens=False, return_tensors="pt")["input_ids"].to("cuda:1")
            outputs = model.generate(input_ids, max_length=target_length, num_return_sequences=1, num_beams=50, output_scores=True, return_dict_in_generate=True)
            output_text = tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            score = torch.exp(outputs.sequences_scores)
            output_texts += output_text
            
            scores.append(score)
        #print(row["pseudoword"], str(output_texts), str([float(score) for score in scores]))
        print(".", end="")
        return pd.Series({"construction": row["construction"], "pseudoword": row["pseudoword"].iloc[0], "orig_example": row["example"], "generated": str(output_texts), "scores": str([float(score) for score in scores])})
    except Exception as e:
        print(":", end="")
        return pd.Series({"construction": row["construction"], "pseudoword": row["pseudoword"].iloc[0], "orig_example": row["example"], "generated": str(e), "scores": "[-1.0]"})

examples_reset = examples.reset_index()
pseudoword_output_scores = examples_reset[["construction", "pseudoword", "example", "cue", "pseudoword"]].progress_apply(complete_cues, axis=1)
pseudoword_output_scores

  0%|          | 0/506 [00:00<?, ?it/s]

..........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

,construction,pseudoword,orig_example,generated,scores
0,5,Und5,"[Und schon gar nicht mit der Mehrwertsteuer .,...","['Und...... ', 'Und...... ', 'Und...... ', 'Un...","[0.7355748414993286, 0.7355748414993286, 0.735..."
1,5,erst5,[Trainer Lucien Favre hatte schon seine beiden...,"['............', 'Tatsächlich sind wir uns nie...","[0.6372720003128052, 0.8877686858177185, 0.634..."
2,5,gar5,[Es hat Afghanistan nicht stabilisiert und sch...,['Es hat Afghanistan nicht stabilisiert und sc...,"[0.7870361804962158, 0.8611741065979004, 0.565..."
3,5,nicht5,[Es hat Afghanistan nicht stabilisiert und sch...,"['Es hat Afghanistan nicht ... ', 'Dass es in ...","[0.7713918685913086, 0.8780519366264343, 0.563..."
4,5,recht5,[Trainer Lucien Favre hatte schon seine beiden...,"['.............', 'Tatsächlich sind wir uns ni...","[0.645953893661499, 0.9032965302467346, 0.5911..."
...,...,...,...,...,...
501,1884,Gold1884,"[Schweigen ist Silber , reden ist Gold ., Schw...","['Schweigen ist Silber, reden ist Gold .. ', '...","[0.8694262504577637, 0.8802860975265503, 0.858..."
502,1884,Silber1884,"[Schweigen ist Silber , reden ist Gold ., Schw...","['Schweigen ist Silber....', 'Schweigen ist Si...","[0.844407320022583, 0.844407320022583, 0.82320..."
503,1884,ist1884,"[Schweigen ist Silber , reden ist Gold ., Schw...","['Schweigen ist.....', 'Schweigen ist.....', '...","[0.8271209597587585, 0.8271209597587585, 0.702..."
504,1986,kaum1986,[Die Vorhut vor 20.000 Jahren war für das Ries...,"['Die Vorhut vor 20.000 Jahren.. ', ', für das...","[0.5729354023933411, 0.5011646151542664, 0.733..."


In [19]:
examples = pseudoword_output_scores[["pseudoword", "generated", "scores"]]

examples

,pseudoword,generated,scores
0,Und5,"['Und...... ', 'Und...... ', 'Und...... ', 'Un...","[0.7355748414993286, 0.7355748414993286, 0.735..."
1,erst5,"['............', 'Tatsächlich sind wir uns nie...","[0.6372720003128052, 0.8877686858177185, 0.634..."
2,gar5,['Es hat Afghanistan nicht stabilisiert und sc...,"[0.7870361804962158, 0.8611741065979004, 0.565..."
3,nicht5,"['Es hat Afghanistan nicht ... ', 'Dass es in ...","[0.7713918685913086, 0.8780519366264343, 0.563..."
4,recht5,"['.............', 'Tatsächlich sind wir uns ni...","[0.645953893661499, 0.9032965302467346, 0.5911..."
...,...,...,...
501,Gold1884,"['Schweigen ist Silber, reden ist Gold .. ', '...","[0.8694262504577637, 0.8802860975265503, 0.858..."
502,Silber1884,"['Schweigen ist Silber....', 'Schweigen ist Si...","[0.844407320022583, 0.844407320022583, 0.82320..."
503,ist1884,"['Schweigen ist.....', 'Schweigen ist.....', '...","[0.8271209597587585, 0.8271209597587585, 0.702..."
504,kaum1986,"['Die Vorhut vor 20.000 Jahren.. ', ', für das...","[0.5729354023933411, 0.5011646151542664, 0.733..."


#### Anzahl neuer Sätze

In [21]:
examples.to_csv(f"../../out/comapp/mbart/data_mbart_vanilla.tsv", sep="\t", decimal=",")
#examples.to_excel(f"../../out/comapp/mbart/data_mbart_vanilla.xlsx")

In [22]:
pseudoword_output_scores.to_csv("../../out/comapp/mbart/data_mbart_vanilla_complete.tsv", sep="\t", decimal=",")